# Knowledge Graph Index 
## Default llama index KG prompt

In [1]:
import os
import time
import pathlib
from pyvis.network import Network

import sys
#kron extensions to llama_index to support openai compatible api
sys.path.append('../llama-index/')

import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

import openai
os.environ['OPENAI_API_KEY'] = "sk-48characterstofakeanopenaikey48charactersopenai0"
os.environ['OPENAI_API_BASE'] = "http://10.0.0.222:30307/v1"
openai.api_key = "sk-48characterstofakeanopenaikey48charactersopenai0"
openai.api_base = "http://10.0.0.222:30307/v1"

model = "Writer/camel-5b-hf"
#model = "mosaicml/mpt-7b-instruct"
#model = "mosaicml/mpt-30b-instruct"

#CORPUS = 'ArxivHealthcareNLP'
CORPUS = 'arxiv_cl'

In [2]:
def load_properties(filepath, sep='=', comment_char='#'):
    '''
    Read the file passed as parameter as a properties file.
    '''
    props = {}
    with open(filepath, "rt") as f:
        for line in f:
            l = line.strip()
            if l and not l.startswith(comment_char):
                key_value = l.split(sep)
                key = key_value[0].strip()
                value = sep.join(key_value[1:]).strip().strip('"') 
                props[key] = value 
    return props

corpus_properties = load_properties(f"corpora/{CORPUS}.properties")
corpus_properties

{'account': '@arxiv_cl@creative.ai',
 'latest': '110803680085564152',
 'corpus_base': '/home/arylwen/datasets/documents/arxiv_cl'}

## Using Knowledge Graph

#### Building the Knowledge Graph

In [3]:

from llama_index import StorageContext
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index import KnowledgeGraphIndex
from llama_index.graph_stores import SimpleGraphStore 
from llama_index import load_index_from_storage 
from llama_index.langchain_helpers.text_splitter import SentenceSplitter
from llama_index.node_parser import SimpleNodeParser

import tiktoken

#extensions to llama_index to support openai compatible endpoints, e.g. llama-api
from kron.llm_predictor.KronOpenAILLM import KronOpenAI
from kron.llm_predictor.KronLLMPredictor import KronLLMPredictor

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [4]:
CORPUS_BASE = corpus_properties['corpus_base']
TXT_BASE = f'{CORPUS_BASE}/text_cleaned/'
PROCESSED_TXT_BASE = f'{CORPUS_BASE}/text_cleaned_out/'
persist_path = f"storage/{model.replace('/', '-')}-default"

#folder to save succesive versions of the pyvis graph
HTML_FOLDER = f"html/{model.replace('/', '-')}-default"
if not os.path.exists(HTML_FOLDER):
    print(f'Creating {HTML_FOLDER}.')
    os.makedirs(HTML_FOLDER)

if not os.path.exists(PROCESSED_TXT_BASE):
    print(f'Creating {PROCESSED_TXT_BASE}.')
    os.makedirs(PROCESSED_TXT_BASE)

In [5]:
#writer/camel uses endoftext 
from llama_index.utils import globals_helper
enc = tiktoken.get_encoding("gpt2")
tokenizer = lambda text: enc.encode(text, allowed_special={"<|endoftext|>"})
globals_helper._tokenizer = tokenizer

In [6]:
documents = SimpleDirectoryReader(TXT_BASE, filename_as_id=True).load_data()

In [7]:
# define LLM
#llm=OpenAI(temperature=0.01, model=model)
llm=KronOpenAI(temperature=0.01, model=model)
#chunk_size+prompt_length+expected length of returned triples must be less than max_tokens
#llm.max_tokens = 274 #128-32
#some sentences can be really long and the text spliter will enter an infinit loop
#llm.max_tokens = 400 #256-64
llm.max_tokens = 384 #192-48
llm_predictor = KronLLMPredictor(llm)
print(llm_predictor.metadata)

context_window=2048 num_output=384 is_chat_model=False model_name='unknown'


In [8]:
# define TextSplitter
text_splitter = SentenceSplitter(chunk_size=192, chunk_overlap=48, paragraph_separator='\n')

In [9]:
#define NodeParser
node_parser = SimpleNodeParser(text_splitter=text_splitter)

In [10]:
#define ServiceContext
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, node_parser=node_parser)

In [11]:
## create pyvis graph
def save_pyvis_network_graph(file_name):
    #display all nodes
    g = index.get_networkx_graph(limit = 6000)
    net = Network(height='1000px', width='100%', notebook=True, cdn_resources="in_line", directed=True)
    net.from_nx(g)
    html_name = f'{HTML_FOLDER}/{file_name}.html'
    print(html_name)
    net.show(html_name)

In [12]:


if os.path.exists(persist_path):
    start = time.time()
    print(f'Loading index from {persist_path}')
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir=persist_path)
    # load index
    index = load_index_from_storage(storage_context=storage_context, 
                                    service_context=service_context, 
                                    max_triplets_per_chunk=2,
                                    show_progress = True)
    ## add documents to index
    #print(index)
    #print(type(documents))
    for d in documents:
        file_name = pathlib.Path(d.id_).name
        print(f'Processing: {file_name}')
        #index the document: extract triples and inseart into the KG graph
        index.insert(document = d)
        #move the file to the processed folder
        in_file_name = f'{TXT_BASE}/{file_name}'
        processed_file_name = f'{PROCESSED_TXT_BASE}/{file_name}'
        pathlib.Path(in_file_name).rename(processed_file_name)
        #index is modified after each doc
        save_pyvis_network_graph(file_name)
        index.storage_context.persist(persist_dir=persist_path)
    end = time.time()
    print(f"Documents added in: {end-start}s")
else:
    print('No KGIndex found, starting fresh.')
    graph_store = SimpleGraphStore()
    storage_context = StorageContext.from_defaults(graph_store=graph_store)

    # NOTE: can take a while! 

    start = time.time()
    index = KnowledgeGraphIndex.from_documents(
        documents,
        max_triplets_per_chunk=2,
        storage_context=storage_context,
        service_context=service_context,
    )
    #move files to the processed files folder/s3/other location
    for d in documents:
        #d.id_ is the full path of the input file
        file_name = pathlib.Path(d.id_).name
        in_file_name = f'{TXT_BASE}/{file_name}'
        processed_file_name = f'{PROCESSED_TXT_BASE}/{file_name}'
        pathlib.Path(in_file_name).rename(processed_file_name)
        save_pyvis_network_graph(file_name)
    #save index TODO what if it fails - compensatory transaction
    index.storage_context.persist(persist_dir=persist_path)
    end = time.time()
    print(f"Knowledge Graph built in: {end-start}s")


Loading index from storage/Writer-camel-5b-hf-default
INFO:llama_index.indices.loading:Loading all indices.
Processing: 2302.08575v1.Foundation_Models_for_Natural_Language_Processing____Pre_trained___Language_Models_Integrating_Media.pdf.txt
html/Writer-camel-5b-hf-default/2302.08575v1.Foundation_Models_for_Natural_Language_Processing____Pre_trained___Language_Models_Integrating_Media.pdf.txt.html
html/Writer-camel-5b-hf-default/2302.08575v1.Foundation_Models_for_Natural_Language_Processing____Pre_trained___Language_Models_Integrating_Media.pdf.txt.html
Processing: 2307.15002v1.Gzip_versus_bag_of_words_for_text_classification_with_KNN.pdf.txt
html/Writer-camel-5b-hf-default/2307.15002v1.Gzip_versus_bag_of_words_for_text_classification_with_KNN.pdf.txt.html
html/Writer-camel-5b-hf-default/2307.15002v1.Gzip_versus_bag_of_words_for_text_classification_with_KNN.pdf.txt.html
Documents added in: 4504.9427263736725s
